In [3]:
import numpy as np

In [44]:
class optimizacion:
    def __init__(self,function):
        self.f = function

    def gradiente(self,x):
        x = x.astype(np.float64)
        h = np.float64(1e-4)
        k = 1/(2*h)
        n = x.shape[0]
        grad = np.zeros(n).astype(np.float64)
        for i in range(n):
            aux1 = np.copy(x)
            aux2 = np.copy(x)
            aux1[i] = aux1[i]+h
            aux2[i] = aux2[i]-h
            grad[i] = self.f(aux1) - self.f(aux2)
            grad[i] = grad[i]*k
        return grad
    
    def hessiana(self,x):
        x = x.astype(np.float64)
        h = np.float64(1e-2)
        k = 1/(h**2)
        n = x.shape[0]
        hess = np.zeros((n,n)).astype(np.float64)
        for i in range(n):
            for j in range(i+1):
                if i == j:
                    aux1 = np.copy(x)
                    aux2 = np.copy(x)
                    aux1[i] = aux1[i]+h
                    aux2[i] = aux2[i]-h
                    hess[i,j] = self.f(aux1) + self.f(aux2) - 2*self.f(x)
                    hess[i,j] = hess[i,j]*k
                else:
                    aux1 = np.copy(x)
                    aux2 = np.copy(x)
                    aux3 = np.copy(x)
                    aux1[i] = aux1[i]+h
                    aux1[j] = aux1[j]+h
                    aux2[i] = aux2[i]+h
                    aux3[j] = aux3[j]+h
                    hess[i,j] = self.f(aux1) - self.f(aux2) - self.f(aux3) + self.f(x)
                    hess[i,j] = hess[i,j]*k
                    hess[j,i] = hess[i,j]
        return hess
    
    def es_optimo(self,x):
        n = x.shape[0] 
        grad = abs(self.gradiente(x))
        eps = np.float64(1e-10) * np.ones(n)
        if all(grad < eps):
            hess = self.hessiana(x)
            if all (np.linalg.eigh(hess)[0] >= 0):
                optimo = True
            else:
                optimo = False
        else:
            optimo = False
        return optimo
    
    def wolfe(self,x,p,alpha,c1,c2):
        aux = x + alpha*p
        producto = self.gradiente(x).dot(p)
        bueno = False
        if self.f(aux) <= self.f(x) + c1*alpha*producto:
            if self.gradiente(aux).dot(p) >= c2*producto:
                bueno = True
        return bueno
    
    def alpha(self,x,p,c1,c2,rho):
        a = 1
        i = 1
        M = 5000
        while i<M and self.wolfe(x,p,a,c1,c2)==False:
            a = rho*a
            i = i+1
        if i==M:
            print("iteraciones maximas alcanzadas en Wolfe")
        return a
    
    def volver_pd(self,x):
        if np.all(np.linalg.eigvals(x) > 0) == True:
            return x
        else:
            e = abs(np.linalg.eigvals(x))
            l = min(e) + 3*np.finfo(float).eps
            E = np.identity(len(x))
            x = x+(l*E)
        return x
    
    def busqueda_lineal_newton(self,x,modificada = False):
        k=0
        c1 = 0.1
        c2 = 0.8
        rho = 0.9
        a = 1
        while k<300 and self.es_optimo(x)==False:
            B = self.hessiana(x)
            g = -self.gradiente(x)
            p = np.linalg.solve(B,g)
            if modificada == True:
                B = self.volver_pd(B)
                a = self.alpha(x,p,c1,c2,rho)
            x = x+a*p
            k=k+1
        if k==300:
            print ("iteraciones maximas alcanzadas en busqueda lineal")
        return x

In [45]:
f = lambda x: (2-x[0])**2 + (3-x[1])**2
opt = optimizacion(f)

In [47]:
x = np.array([0,0])
opt.busqueda_lineal_newton(x)

array([2., 3.])